In [1]:
#imports and data loading
import pandas as pd

from random import sample
from utils.ccf_utils import get_ccf_ids, get_ccf_property #, get_connectivity_matrix
from utils.skeleton_loading_utils import number_of_samples, load_lm_datasets, progress_bar
import matplotlib.pyplot as plt
import numpy as np
import skeleton_plot as skelplot
import seaborn as sns
import random
import trimesh
import k3d
from collections import defaultdict
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from brainglobe_atlasapi import BrainGlobeAtlas
import brainglobe_atlasapi as atlasapi
%matplotlib inline

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
This cell sets up a variable called `data_root` that you should use in any code below to access the dataset in question (e.g. paths to manifest files for the SDK should be made relative to this variable).
</div>

In [2]:
import platform
platstring = platform.platform()

if 'Darwin' in platstring:
    # macOS 
    data_root = "/Volumes/Brain2024/anatomy/"
elif 'Windows'  in platstring:
    # Windows (replace with the drive letter of USB drive)
    data_root = "E:/anatomy/"
elif ('amzn2' in platstring):
    # then on AWS
    data_root = "/data/"
else:
    # then your own linux platform
    # EDIT location where you mounted hard drive
    data_root = "/media/$USERNAME/Brain2024/anatomy/"

In [3]:
# Load ccf atlas
atlasapi.config.write_config_value('brainglobe_dir', f'{data_root}/.brainglobe')
ccf_atlas = BrainGlobeAtlas('allen_mouse_10um')

skel_list = load_lm_datasets(data_root)

Loading datasets...
exaSPIM_609281_2022-11-03_13-49-18_reconstructions/precomputed:  [==================================================] 25/25
exaSPIM_651324_2023-03-06_15-13-25_reconstructions/precomputed:  [==================================================] 13/13
exaSPIM_653158_2023-06-01_20-41-38_reconstructions/precomputed:  [==================================================] 42/42
exaSPIM_653980_2023-08-10_20-08-29_reconstructions/precomputed:  [==================================================] 20/20
mouselight_reconstructions/precomputed:  [==================================================] 1549/1549


In [4]:
#it may be useful to copy some functions from the demo notebook here functions


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<font size="4"><p><b>Exercise 1:</b> Let's look at another region: VAL (ccf id = 629). Find all neurons sending output to this region and plot them (axon end points) 
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<font size="4"><p><b>Exercise 2:</b> Pick one neuron here. Plot it in 3D with the CCF mesh. What fraction of this neuron's axonal endpoints are in our original target region of VAL? Look at the report and bar graph. Does VAL seem to be this neuron's "primary" projection target?
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<font size="3"><p><b>Exercise 3:</b> Find all neurons with somas in VAL and plot them.
    
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<font size="4"><p><b>Exercise 4:</b> Calculate the feature matrix of the projection targets of all these cells.
    
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<font size="3"><p><b>Exercise 5:</b> Calculate the following features for all neurons: total axon length, total dendrite length and number of branch points. (Hint: Look at supplemental information)
    
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<font size="3"><p><b>Exercise 6:</b> How do these features correlate against each other? How do they correlate against the number of inputs to a certain region?
    
    
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<font size="3"><p><b>Extra Credit:</b> Cluster the cells with your choice of k. What do you observe? 
    
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
    
<h4>Extra Exercises </h4>
    
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

<font size="4"><p><b>Task 1.1:</b>  Find the most fine-grained brain region contained in the CCF (i.e. region at the largest depth).
    
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

<font size="4"><p><b>Task 1.2:</b>  How many brain regions are at a depth=0, 1,..., n where n is the depth value from Task 1.1.

</div>